In [27]:
import pandas as pd

df = pd.read_csv("stackoverflow_perguntas.csv")
df.sample(10)

,Perguntas,Tags
515,Tenho montado a seguinte estrutura em HTML: C...,jquery html
572,"Como fazer um efeito em jQuery, que quando use...",jquery html
183,Estou usando o plugin jQuery Validate para val...,html
1404,Estou fazendo um template e preciso colocar a ...,html
1187,Estou criando um JS para executar o código com...,jquery html
4309,"No PHP, podemos converter uma quebra de linha ...",html
1312,Estou migrando um projeto e o mesmo possuía al...,angular
5009,Boa noite! Estou tentando consumir uma API uti...,jquery
2659,Estive a fazer uma pesquisa em PT e EN sobre c...,html
4691,"Por exemplo, tenho uma CODE dentro de uma var...",html


In [28]:
print(len(df))

5408


In [29]:
df.Tags.unique()

array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

In [30]:
print(len(df.Tags.unique()))

37


In [31]:
lista_de_tags = list()
for tags in df.Tags.unique():
    for tag in tags.split():
        if tag not in lista_de_tags:
            lista_de_tags.append(tag)
print(lista_de_tags)

['node.js', 'jquery', 'html', 'angular']


In [32]:
def nova_coluna(lista_tag, dataframe, nome_tags):
    for tag in lista_tag:
        coluna = list()
        for linha_tag in dataframe[nome_tags]:
            if tag in linha_tag:
                coluna.append(1)
            else:
                coluna.append(0)
        dataframe[tag] = coluna

nova_coluna(lista_de_tags, df, "Tags")
df.sample(10)

,Perguntas,Tags,node.js,jquery,html,angular
268,A pagina tem um datatables https://datatables....,jquery,0,1,0,0
1908,"Olá estou com um problema aqui, eu preciso usa...",node.js,1,0,0,0
1843,estou com o seguinte código JS: Ele esta aplic...,jquery,0,1,0,0
5225,"Amigos, tenho uma questão que ficaria feliz se...",html,0,0,1,0
3134,CODE conseguem identificar? não é desenhado ...,html,0,0,1,0
1037,"Olá, estou aprendendo Javascript e tenho uma d...",html,0,0,1,0
2979,Fiz a ativação tudo certo do jQuery datatable ...,jquery,0,1,0,0
1447,"Quando eu dou um npm install no meu projeto, e...",angular node.js,1,0,0,1
2744,Boa tarde a todos! Galera tenho uma página per...,jquery,0,1,0,0
2665,Tenho um SVG rasteirizado e inline em uma pagi...,html,0,0,1,0


In [33]:
lista_zip_tags = list(zip(
    df[lista_de_tags[0]],
    df[lista_de_tags[1]],
    df[lista_de_tags[2]],
    df[lista_de_tags[3]]
))
df["todas_tags"] = lista_zip_tags
df.sample(10)

,Perguntas,Tags,node.js,jquery,html,angular,todas_tags
5081,"Olá, Através da função abaixo, pretendo exibi...",jquery,0,1,0,0,"(0, 1, 0, 0)"
5084,Preciso fazer um gráfico como esse Mas não e...,jquery html,0,1,1,0,"(0, 1, 1, 0)"
4299,CODE Desta maneira ele esta carregando a ima...,jquery,0,1,0,0,"(0, 1, 0, 0)"
2979,Fiz a ativação tudo certo do jQuery datatable ...,jquery,0,1,0,0,"(0, 1, 0, 0)"
1084,Pretendo criar um Serach Bar na minha home est...,angular,0,0,0,1,"(0, 0, 0, 1)"
4352,Eu estou desenvolvendo uma aplicação web para ...,jquery,0,1,0,0,"(0, 1, 0, 0)"
2090,Gostaria de obter a conexão do arquivo dbConne...,node.js,1,0,0,0,"(1, 0, 0, 0)"
4252,Estou utilizando a versão v0.10.37 do Node e a...,node.js,1,0,0,0,"(1, 0, 0, 0)"
5344,"No site que estou desenvolvendo, gostaria de c...",html,0,0,1,0,"(0, 0, 1, 0)"
4554,Tenho um autocomplete que funciona perfeitamen...,jquery,0,1,0,0,"(0, 1, 0, 0)"


In [34]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    df.Perguntas,
    df.todas_tags,
    test_size = 0.2,
    random_state = 123
)

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorize = TfidfVectorizer(max_features=5000, max_df=0.85)


In [36]:
vectorize.fit(df.Perguntas)
x_train_tfidf = vectorize.transform(x_train)
x_test_tfidf = vectorize.transform(x_test)

In [37]:
print(x_train_tfidf.shape)
print(x_test_tfidf.shape)

(4326, 5000)
(1082, 5000)


In [38]:
import numpy as np

y_train_array = np.asarray(list(y_train))
y_test_array = np.asarray(list(y_test))

In [40]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

regressao_logistica = LogisticRegression()
classificador_onevsrest = OneVsRestClassifier(regressao_logistica)
classificador_onevsrest.fit(x_train_tfidf,y_train_array)
result_onevsrest = classificador_onevsrest.score(x_test_tfidf,y_test_array)
print("Resultado {0: .2f}%".format(result_onevsrest*100))

Resultado  41.68%


In [41]:
from sklearn.metrics import hamming_loss

previsao_onevsrest = classificador_onevsrest.predict(x_test_tfidf)
hamming_loss_onevsrest = hamming_loss(y_test_array,previsao_onevsrest)
print("Hamming Loss {0: .2f}".format(hamming_loss_onevsrest))

Hamming Loss  0.19


In [42]:
df.corr()

,node.js,jquery,html,angular
node.js,1.000000,-0.321485,-0.273523,-0.101787
jquery,-0.321485,1.000000,-0.253977,-0.366269
html,-0.273523,-0.253977,1.000000,-0.286706
angular,-0.101787,-0.366269,-0.286706,1.000000


In [44]:
#!pip install scikit-multilearn

You should consider upgrading via the 'c:\users\pedro\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [47]:
from skmultilearn.problem_transform import ClassifierChain

classificador_cadeia = ClassifierChain(regressao_logistica)
classificador_cadeia.fit(x_train_tfidf,y_train_array)
result_cadeia = classificador_cadeia.score(x_test_tfidf,y_test_array)

previsao_cadeia = classificador_cadeia.predict(x_test_tfidf)
hamming_loss_cadeia = hamming_loss(y_test_array,previsao_cadeia)

print("Hamming Loss {0: .2f}".format(hamming_loss_cadeia))

print("Resultado {0: .2f}%".format(result_cadeia*100))

Hamming Loss  0.21
Resultado  49.82%


In [48]:
from skmultilearn.problem_transform import BinaryRelevance

classificador_br = BinaryRelevance(regressao_logistica)
classificador_br.fit(x_train_tfidf,y_train_array)
result_br = classificador_br.score(x_test_tfidf,y_test_array)

previsao_br = classificador_br.predict(x_test_tfidf)
hamming_loss_br = hamming_loss(y_test_array,previsao_br)

print("Hamming Loss {0: .2f}".format(hamming_loss_br))

print("Resultado {0: .2f}%".format(result_br*100))

Hamming Loss  0.19
Resultado  41.68%


In [49]:
from skmultilearn.adapt import MLkNN

classificador_mlknn = MLkNN()

classificador_mlknn.fit(x_train_tfidf,y_train_array)
result_mlknn = classificador_mlknn.score(x_test_tfidf,y_test_array)

previsao_mlknn = classificador_mlknn.predict(x_test_tfidf)
hamming_loss_mlknn = hamming_loss(y_test_array,previsao_mlknn)

print("Hamming Loss {0: .2f}".format(hamming_loss_mlknn))

print("Resultado {0: .2f}%".format(result_mlknn*100))

C:\Users\Pedro\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_neighbors=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Hamming Loss  0.25
Resultado  32.53%


In [56]:
resultados_classificacao = pd.DataFrame()
resultados_classificacao["perguntas"] = x_test.values
resultados_classificacao["tags real"] = list(y_test)
resultados_classificacao["BR"] = list(previsao_br.toarray())
resultados_classificacao["cadeia"] = list(previsao_cadeia.toarray())
resultados_classificacao["mlknn"] = list(previsao_mlknn.toarray())
resultados_classificacao

,perguntas,tags real,BR,cadeia,mlknn
0,estou com conflito entre o CODE e os CODE ...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 0]"
1,Estou fazendo um site que eu sou obrigado a us...,"(0, 0, 1, 0)","[0, 0, 1, 0]","[0.0, 0.0, 1.0, 0.0]","[0, 1, 1, 0]"
2,Recentemente fiz um refactor do meu código par...,"(1, 0, 0, 0)","[1, 0, 0, 0]","[1.0, 0.0, 0.0, 0.0]","[1, 0, 0, 0]"
3,Eu tenho esse código em CODE que passo valore...,"(0, 1, 1, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 1, 0]"
4,"Olá, em minha função tem o evento CODE que de...","(0, 1, 1, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 1, 0]"
...,...,...,...,...,...
1077,Estou a desenvolver um website em jQuery. E at...,"(0, 1, 0, 0)","[0, 1, 1, 0]","[0.0, 1.0, 1.0, 0.0]","[0, 0, 1, 0]"
1078,Estou usando este plugin - jquery autocomplete...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]"
1079,"Tenho o seguinte jQuery: CODE Nisto, quanti...","(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]"
1080,Estou usando o SimpleModal Contact Form de Eri...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]"
